In [14]:
from astropy.io import fits
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
# Detect image shift https://scikit-image.org/docs/dev/auto_examples/registration/plot_register_translation.html
from skimage.registration import phase_cross_correlation
# Shift the image
from scipy.ndimage import shift
import pandas as pd
import os
cal_dir = Path(os.environ['DATA'], 'DDS', 'Taka', 'Calibration')
rgbdirs = [Path(os.environ['DATA'], 'DDS', 'Taka', 'Lights', 'IC_1805', c) for c in 'RGB']
for cdir in rgbdirs: print(cdir.exists())
rgbf = [sorted(rgbdir.rglob('*.fit')) for rgbdir in rgbdirs]
print(len(rgbf[0]), len(rgbf[1]), len(rgbf[2]))

def fitsread(f, header=False):
    with fits.open(f) as hdul:
        data = hdul[0].data
        if header:
            h = hdul[0].header
            return data, h
        else:
            return data

True
True
True
18 8 15


In [21]:
# Masters
dark300f = Path(cal_dir, 'masterDark300_bin2_median_Jan_2019.fits')
dark600f = Path(cal_dir, 'masterDark600_bin1_median_Jan_2019.fits')

RFlatf = Path(cal_dir, 'masterRFlat_bin2_Jan_26_2019.fits')
GFlatf = Path(cal_dir, 'masterGFlat_bin2_Feb_3_2019.fits')
BFlatf = Path(cal_dir, 'masterBFlat_bin2_Feb_4_2019.fits')

mDark = fitsread(dark300f)

# RED

In [24]:
# Load flat field
cFlat = fitsread(RFlatf)

In [19]:
clights = np.array([fitsread(f) for f in rgbf[0]])

In [23]:
cCal1 = clights - mDark
cCal1[cCal1 < 0] = 0

In [25]:
cCal2 = cCal1 / cFlat

In [26]:
scale = np.median(cCal1) / np.median(cCal2)
print(scale)

35218.82377664365


In [ ]:
Hregdir = Path(lights_dir, 'H', 'registered')
Hregdir.mkdir(parents=False, exist_ok=True)

fov = np.s_[1000:1000+1024, 1000:1000+1024]
ref = HCal2[int(len(Hf)/2)][fov]

imshifts = []
for i, f in enumerate(Hf):
    image = HCal2[i][fov]
    imshift, _, _ = phase_cross_correlation(ref, image, upsample_factor=10)
    imshifts.append(imshift)
    if (imshift[0]==0) & (imshift[1]==0):
        print(f'no shift at file {f}')
        image2 = HCal2[i].copy()*scale
    else:
        image2 = shift(HCal2[i], imshift)*scale
    
    fits.writeto(Path(Hregdir, f'{f.stem}.reg.fits'), image2.astype(np.float32), overwrite=True)

In [ ]:
Hregf = sorted(Hregdir.rglob('*.reg.fits'))
len(Hregf)

In [ ]:
Hreg = np.array([fitsread(f) for f in Hregf])

In [ ]:
Hstackmed = np.median(Hreg, axis=0)

In [ ]:
fits.writeto(Path(lights_dir, 'stacked', 'Hstackmed.fits'), Hstackmed.astype(np.float32), overwrite=True)

In [ ]:
plt.figure(figsize=(20,13))
plt.imshow(Hstackmed, vmin=np.percentile(Hstackmed, 0.1), vmax=np.percentile(Hstackmed, 99.9), origin='lower', cmap='gray')
plt.tick_params(
    axis='both',        
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    left=False,
    labelleft=False) 
plt.tight_layout()
plt.savefig('Hstackmed2.jpg')